In [1]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
DATA_PATH = r'C:\Users\connor\PycharmProjects\trading\data\analytics\analytics_voo.csv'
df = pd.read_csv(DATA_PATH, low_memory=False)

target_column = 'target'
non_target_columns = ['open', 'high', 'low', 'close', 'volume', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema']

df_cols = df[non_target_columns]

In [4]:
df_cols.shape[1]

17

In [ ]:
n_splits = 5
tscv  = TimeSeriesSplit(n_splits)

scores = []

lstm = nn.LSTM(df_cols.shape[1], hidden_size=(df_cols.shape[1] + 1)/2, num_layers=2, batch_first=True)  # Input dim is the number of columns in df_cols. output is the mean between the number of input dimensions and the final output dimension, which is just a binary 1,0. Read a paper that said hidden units performs best with a depth between the output and input dimension size.

for i, (train_index, test_index) in enumerate(tscv.split(df)): 
    # split data for each time series partition 
    y_train = df[target_column].iloc[train_index]
    X_train = df[non_target_columns].iloc[train_index]
    y_test = df[target_column].iloc[test_index]
    X_test = df[non_target_columns].iloc[test_index]
    
    inputs = [torch.tensor(X_train.values)]  # convert inputs to tensors

In [5]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [6]:
inputs

[tensor([[-1.1231,  1.0100, -0.6803]]),
 tensor([[ 0.3528, -2.0210,  1.5867]]),
 tensor([[ 1.2560, -0.9985,  1.4938]]),
 tensor([[ 2.0762,  0.1923, -0.0624]]),
 tensor([[-0.7145,  0.4195, -0.4065]])]